In [2]:
%pip install pandas
%pip install fuzzywuzzy
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sqlite3
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

In [4]:
con = sqlite3.connect("db.db")

In [5]:
sr = pd.read_sql_query("SELECT * from soundrecording", con).set_index("id")

In [6]:
sr

,sr_id,title,artists,isrcs,contributors
id,,,,,
1,spotify_apidsr__2NbYAPqE6FTyQte9kW4vgr,Astronomia - Tequila Edit,Bing Lee,ITZB42033435,Edizioni Lungoviaggio|Victor Pool|Ruben Christ...
2,crawler_believe__26052217,Astronomia (feat. Tish),"Marco Marzi, Marco Skarica, David White",ITF341800025,Игумнов
3,crawler_believe__34028360,Astronomia (Coffin Dance) [Dance Edit],Josh Nor,FR2X42061192,Victor Pool|Ruben den Boer|Антон Игумнов
4,crawler_believe__34168410,Astronomia (Coffin Dance) [Tequila Edit],Josh Nor,FR96X2013991,Victor Pool|Ruben den Boer|Антон Игумнов
5,crawler_fuga__7427128907609_1_6_ITZB42136782,Astronomia (Purple Mix),Josh Nor,ITZB42136782,Ruben Christopher Den Boer|Anton Igumnov|Victo...
...,...,...,...,...,...
55545,spotify_apidsr__3x99UdcqjXhQcqdgadKeXA,Belladoona,Masahiko Sato,GBRNP1400106,M. Sato
55546,youtube_dsr__A219026358613851,BELLADOONA,MASAHIKO SATO,GBRNP1400106,SATOU MASAHIKO
55547,spotify__3x99UdcqjXhQcqdgadKeXA,Belladoona,Masahiko Sato,GBRNP1400106,M. Sato


In [7]:
groundtruth = pd.read_csv("groundtruth.csv")
groundtruth

,q_source_id,m_source_id,tag
0,spotify_apidsr__2NbYAPqE6FTyQte9kW4vgr,crawler_believe__26052217,invalid
1,crawler_believe__34028360,crawler_believe__34168410,valid
2,crawler_fuga__7427128907609_1_6_ITZB42136782,crawler_believe__42573832,valid
3,crawler_believe__34168476,spotify_apidsr__3kOHtCewbmdWgMVgJ8rpkC,invalid
4,spotify_apidsr__28JA0VuEMS8i3N6fpRXr2M,spotify_apidsr__1d6j1PD3Z8NqbCgCYKDbCy,invalid
...,...,...,...
28366,apple__1354975784,youtube_dsr__A461439239803827,valid
28367,apple__1052537885,crawler_pias__5060099505690_1_2_GBRNP1400106,valid
28368,crawler_247__5060099505690_GBRNP1400106,spotify_apidsr__3x99UdcqjXhQcqdgadKeXA,valid
28369,youtube_dsr__A219026358613851,spotify__3x99UdcqjXhQcqdgadKeXA,valid


In [8]:
dataset = (groundtruth
           .set_index("q_source_id")
           .join(sr.rename(columns={"sr_id": "q_source_id"}).set_index("q_source_id"))
           .rename(columns={"title": "q_title", "artists": "q_artists", "isrcs": "q_isrcs", "contributors": "q_contributors"})
           .reset_index()
           .set_index("m_source_id")
           .join(sr.rename(columns={"sr_id": "m_source_id"}).set_index("m_source_id"))
           .rename(columns={"title": "m_title", "artists": "m_artists", "isrcs": "m_isrcs", "contributors": "m_contributors"})
           .reset_index()
          )
    

In [9]:
dataset

,m_source_id,q_source_id,tag,q_title,q_artists,q_isrcs,q_contributors,m_title,m_artists,m_isrcs,m_contributors
0,amazon_dsr__AB00QLM7UVI,crawler_247__3614591466960_FR10S1541945,invalid,Centuries,Jagerbombs,FR10S1541945,,Centuries,Fall Out Boy,USUM71412644,Jake Sinclair|Michael J. Fonesca|Justin Drew T...
1,amazon_dsr__AB07CNFFC62,spotify_apidsr__73F87Sqh6jQWucOOvz1WFx,valid,Genius,"Sia, Diplo, Labrinth, LSD",USQX91800798,Thomas Wesley Pentz|Timothy Mckenzie|Philip Me...,Genius,"LSD [feat. Sia, Diplo & Labrinth], LSD [feat ...",USQX91800836,"Thomas Pentz|Thomas Pentz, Jr.|Philip Mecksepe..."
2,amazon_dsr__AB07JBMWR4S,youtube_dsr__A988499812004440,invalid,FALLING,R3YAN & BENLON,TCAFK2148987,KIM CANDILORA|RYAN VOJTESAK|MARTIN KOTTMEIER|D...,Falling,TREVOR DANIEL,USUYG1221109,TREVOR DANIEL
3,amazon_dsr__AB07LDDZ1XG,apple__1541884162,valid,Fire on Fire,Sam Smith,GBUM71807533,Steve Mac|Sam Smith,"Fire On Fire (From ""Watership Down"")",Sam Smith,GBUM71807533,Steve Mac|Sam Smith
4,amazon_dsr__AB07ZKWPQBX,crawler_rfa_ndr__ndrhfdb1-11806275,valid,Forever,Lewis Capaldi,,Lewis Capaldi|Sean Douglas|Joe Janiak,Forever (Amazon Original),Lewis Capaldi,DEUM71906191,Lewis Capaldi
...,...,...,...,...,...,...,...,...,...,...,...
28366,youtube_dsr__xklJ8z_4e4w,crawler_emidigitalcontent__00610018948203_USSY...,invalid,Dream Weaver (Made Popular By Gary Wright) [Vo...,Party Tyme Karaoke,USSYS1102469,Gary Wright,DREAM WEAVER (VIOLIN AND GRAND PIANO),STEVEN HALPERN,US2V99710284,
28367,youtube_dsr__xmAw29prKKE,crawler_gdeasia__597175,valid,Angelina,Lou Bega,DEC730100131,,ANGELINA,LOU BEGA,DEC730100131,WEBENAU VON|KLEIST|PLETSCHACHER|[CA] KLEIST|LU...
28368,youtube_dsr__y6ohky8p2HM,spotify_apidsr__4Ly4FpM4zZELvAWrVjFez4,invalid,Fanfare Overture (Live),"Jonne Valtonen, Tokyo Philharmonic Orchestra, ...",SEWDL6139115,Jonne Valtonen,TWENTIETH CENTURY FOX FANFARE,ALFRED NEWMAN,USAR10400309,
28369,youtube_dsr__zPoEDu8dlxI,crawler_believe__33249194,valid,Tip-Toe Through the Tulips With Me,Ambrose and His Orchestra,DEKB71942669,Joe Burke,TIP-TOE THROUGH THE TULIPS WITH ME,AMBROSE AND HIS ORCHESTRA,NLB150900028,


In [10]:
fuzz.ratio("Jagerbombs", "Sia, Jagerbombs")

80

In [11]:
def conicidence(word1, word2):
    if word1 == word2:
        return 1
    return 0

In [12]:
def y_tag(tag):
    if tag == "valid":
        return 1
    if tag == "invalid":
        return 0

In [13]:
dataset["title_sim"] = dataset.apply(lambda x: fuzz.ratio(x.q_title, x.m_title), axis=1)

In [14]:
dataset["artists_sim"] = dataset.apply(lambda x: fuzz.ratio(x.q_artists, x.m_artists), axis=1)

In [15]:
dataset["contributors_sim"] = dataset.apply(lambda x: fuzz.ratio(x.q_contributors, x.m_contributors), axis=1)

In [16]:
dataset["isrcs_coincidence"] = dataset.apply(lambda x: conicidence(x.q_isrcs, x.m_isrcs), axis=1)

In [17]:
dataset["y"] = dataset.apply(lambda x: y_tag(x.tag), axis=1)

In [18]:
dataset.sort_values(["title_sim", "artists_sim"], ascending=False)

,m_source_id,q_source_id,tag,q_title,q_artists,q_isrcs,q_contributors,m_title,m_artists,m_isrcs,m_contributors,title_sim,artists_sim,contributors_sim,isrcs_coincidence,y
5,amazon_dsr__AB07ZQG4R4F,tedald_spotify__6tBDDLLwzrQ5LhDaIox0gl,valid,Before You Go,Lewis Capaldi,DEUM71905868,,Before You Go,Lewis Capaldi,DEUM71905868,Lewis Capaldi,100,100,0,1,1
7,amazon_dsr__B001BJEI1Q,crawler_acinpro__bc6b06ea3775f047189531a465b6af83,valid,Castaways,The Backyardigans,USA820500093,,Castaways,The Backyardigans,USA820500093,Douglas Wieselman|McPaul Smith|The Backyardiga...,100,100,0,1,1
10,amazon_dsr__B001F34X04,spotify__4RAOI1etsgbh5NP3T5R8rN,valid,I Don't Love You,My Chemical Romance,USRE10602907,My Chemical Romance,I Don't Love You,My Chemical Romance,USRE10602907,My Chemical Romance|Chris Lord-Alge|Frank Iero...,100,100,16,1,1
13,amazon_dsr__B001L29H4Q,crawler_247__886445003871_USJI10801081,valid,Circus,Britney Spears,USJI10801081,,Circus,Britney Spears,USJI10801081,"Lukasz ""Doctor Luke"" Gottwald|Dr. Luke|Britney...",100,100,0,1,1
21,amazon_dsr__B00CVGIOHM,crawler_cdbaby__10745843,valid,Satisfya,Imran Khan,GBB291300001,Imran Khan,Satisfya,Imran Khan,GBB291300001,Imran Khan,100,100,100,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27712,youtube_dsr__A848389068442529,crawler_eolasia__10290,invalid,一起走過的日子,劉德華,HKD240000032,,YI QI ZUO GUO DE RI ZI,AARON KWOK,HKI490102117,WU WAI LAP|小美,0,0,0,0,0
27894,youtube_dsr__A898493819985175,crawler_247__825646272440_HKC841400010,invalid,Yan Bu You Zhong,Andy Lau,HKC841400010,,暗里着迷,杨优秀,HKG732116599,CHUN KEUNGLAM|YU RUILIU|JIA LIANGXU,0,0,0,0,0
27914,youtube_dsr__A904080139975896,apple__1370958376,valid,虹之间,金貴晟,CNE041400038,Zheng Wei|金貴晟|Gua-Guo Da-Feng|Jia Gao,BETWEEN THE RAINBOW,JOSHUA JIN,CNE041400038,韦正|大风刮过|金贵晟,0,0,12,1,1
28073,youtube_dsr__A940659983557430,crawler_uniqueopia_ftp__A10409P0000001F6X1,invalid,我们不一样,阿哲,DEHY12102413,高进,WO MEN BU YI YANG,FMP (JESS CHEN XIN LIN),MYFM11800055,,0,0,0,0,0


In [19]:
ds = dataset[["m_source_id", "q_source_id", "title_sim", "artists_sim", "contributors_sim", "isrcs_coincidence", "y"]].set_index(["m_source_id", "q_source_id"])

In [20]:
ds

,,title_sim,artists_sim,contributors_sim,isrcs_coincidence,y
m_source_id,q_source_id,,,,,
amazon_dsr__AB00QLM7UVI,crawler_247__3614591466960_FR10S1541945,100,18,0,0,0
amazon_dsr__AB07CNFFC62,spotify_apidsr__73F87Sqh6jQWucOOvz1WFx,100,52,41,0,1
amazon_dsr__AB07JBMWR4S,youtube_dsr__A988499812004440,14,37,24,0,0
amazon_dsr__AB07LDDZ1XG,apple__1541884162,46,100,100,1,1
amazon_dsr__AB07ZKWPQBX,crawler_rfa_ndr__ndrhfdb1-11806275,44,100,52,0,1
...,...,...,...,...,...,...
youtube_dsr__xklJ8z_4e4w,crawler_emidigitalcontent__00610018948203_USSYS1102469_R29,21,12,0,0,0
youtube_dsr__xmAw29prKKE,crawler_gdeasia__597175,12,38,0,1,1
youtube_dsr__y6ohky8p2HM,spotify_apidsr__4Ly4FpM4zZELvAWrVjFez4,12,5,0,0,0


In [21]:
X = ds[["title_sim", "artists_sim", "contributors_sim", "isrcs_coincidence"]]
y = ds[["y"]]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
X_test

,,title_sim,artists_sim,contributors_sim,isrcs_coincidence
m_source_id,q_source_id,,,,
spotify_apidsr__43a0zndj9PKOZY7bJ6Z7MR,youtube_dsr__A814771753369564,43,25,12,1
spotify_apidsr__2UNke6TriCgvLccPmYUiRQ,crawler_empire_ftp__888915925447_1_2_USUYG1257926,100,91,100,0
apple__1524808074,youtube_dsr__A945231354534201,14,9,0,0
spotify_apidsr__4T3G9kwx4oGgWbEI6GqvPI,crawler_fuga__7330065154114_1_1_SE5IB2206689,100,24,60,0
crawler_new247__US4K30600024,crawler_warnerdigitalcontent__190295323141_1_13_ITV951900009,100,19,0,0
...,...,...,...,...,...
spotify_apidsr__7N1SthvfHhGhV9C9pkPGIa,crawler_warnerdigitalcontent__5054197201295_1_20_USAT22103652,100,0,44,0
spotify_apidsr__5ZzbaEl28x5epxXB7GJIA9,crawler_eolasia__386193,54,18,100,0
crawler_believe__43789748,crawler_believe__30367209,67,40,16,0


In [32]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

C:\Users\mauma\AppData\Local\Temp\ipykernel_2640\2452461117.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=2, random_state=0)

In [33]:
import pickle

filename = "model.pickle"

# save model
pickle.dump(clf, open(filename, "wb"))

# load model
loaded_model = pickle.load(open(filename, "rb"))

In [48]:
import numpy as np
X_sample = np.array([1,2,3,4]).reshape(1, -1)

In [50]:
y_pred = clf.predict(X_sample)[0]
y_pred

C:\Users\mauma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1

In [27]:
y_pred = clf.predict(X_test)

In [28]:
accuracy_score(y_test, y_pred)

0.9418989640072626

In [29]:
X_test

,,title_sim,artists_sim,contributors_sim,isrcs_coincidence
m_source_id,q_source_id,,,,
spotify_apidsr__43a0zndj9PKOZY7bJ6Z7MR,youtube_dsr__A814771753369564,43,25,12,1
spotify_apidsr__2UNke6TriCgvLccPmYUiRQ,crawler_empire_ftp__888915925447_1_2_USUYG1257926,100,91,100,0
apple__1524808074,youtube_dsr__A945231354534201,14,9,0,0
spotify_apidsr__4T3G9kwx4oGgWbEI6GqvPI,crawler_fuga__7330065154114_1_1_SE5IB2206689,100,24,60,0
crawler_new247__US4K30600024,crawler_warnerdigitalcontent__190295323141_1_13_ITV951900009,100,19,0,0
...,...,...,...,...,...
spotify_apidsr__7N1SthvfHhGhV9C9pkPGIa,crawler_warnerdigitalcontent__5054197201295_1_20_USAT22103652,100,0,44,0
spotify_apidsr__5ZzbaEl28x5epxXB7GJIA9,crawler_eolasia__386193,54,18,100,0
crawler_believe__43789748,crawler_believe__30367209,67,40,16,0


In [30]:
X_test["y"] = y_test
X_test["y_pred"] = y_pred

In [31]:
X_test

,,title_sim,artists_sim,contributors_sim,isrcs_coincidence,y,y_pred
m_source_id,q_source_id,,,,,,
spotify_apidsr__43a0zndj9PKOZY7bJ6Z7MR,youtube_dsr__A814771753369564,43,25,12,1,1,1
spotify_apidsr__2UNke6TriCgvLccPmYUiRQ,crawler_empire_ftp__888915925447_1_2_USUYG1257926,100,91,100,0,1,1
apple__1524808074,youtube_dsr__A945231354534201,14,9,0,0,0,0
spotify_apidsr__4T3G9kwx4oGgWbEI6GqvPI,crawler_fuga__7330065154114_1_1_SE5IB2206689,100,24,60,0,0,0
crawler_new247__US4K30600024,crawler_warnerdigitalcontent__190295323141_1_13_ITV951900009,100,19,0,0,0,0
...,...,...,...,...,...,...,...
spotify_apidsr__7N1SthvfHhGhV9C9pkPGIa,crawler_warnerdigitalcontent__5054197201295_1_20_USAT22103652,100,0,44,0,0,0
spotify_apidsr__5ZzbaEl28x5epxXB7GJIA9,crawler_eolasia__386193,54,18,100,0,0,0
crawler_believe__43789748,crawler_believe__30367209,67,40,16,0,0,0


In [58]:
X_test.join(dataset.set_index(["m_source_id", "q_source_id", "title_sim", "artists_sim", "contributors_sim", "isrcs_coincidence", "y"]))

,,,,,,,title_sim,artists_sim,contributors_sim,isrcs_coincidence,y,y_pred,tag,q_title,q_artists,q_isrcs,q_contributors,m_title,m_artists,m_isrcs,m_contributors
m_source_id,q_source_id,title_sim,artists_sim,contributors_sim,isrcs_coincidence,y,,,,,,,,,,,,,,,
amazon_dsr__AB07LDDZ1XG,apple__1541884162,46,100,100,1,1,46,100,100,1,1,1,valid,Fire on Fire,Sam Smith,GBUM71807533,Steve Mac|Sam Smith,"Fire On Fire (From ""Watership Down"")",Sam Smith,GBUM71807533,Steve Mac|Sam Smith
amazon_dsr__AB084TZC83X,itunes_dsr__1627575411,25,50,6,0,0,25,50,6,0,0,0,invalid,ON,BTS,QM4TX2001949,"Suga|Melanie Joy Fontana|Jhope|""Antonina Armat...",ON (Feat. Sia),BTS & Sia,QM4TX2001958,BTS|BTS & SIA
amazon_dsr__B001BJEI1Q,crawler_acinpro__bc6b06ea3775f047189531a465b6af83,100,100,0,1,1,100,100,0,1,1,1,valid,Castaways,The Backyardigans,USA820500093,,Castaways,The Backyardigans,USA820500093,Douglas Wieselman|McPaul Smith|The Backyardiga...
amazon_dsr__B001SML2PQ,crawler_247__86554219,58,40,0,1,1,58,40,0,1,1,1,valid,It's Tricky (Album Version),Run D.M.C.,USAR18600005,,It's Tricky,"RUN-DMC, Run DMC, RUN DMC",USAR18600005,Darryl McDaniels|Russell Simmons|Joseph Ward S...
amazon_dsr__B00JPOY6N4,youtube_dsr__A363504268767711,4,25,3,0,0,4,25,3,0,0,0,invalid,MONSTERS (ALEX S REMIX) [FEAT. ALEX S.],ALEX S.,TCACD1523524,CA SHAPIRO CALEB|CA LERVIK MADS VATHNE|CA RESN...,Monsters,Katie Sky,GB8MX1400041,Katie Sky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youtube_dsr__qzCIBDjEXCc,crawler_247__829410881807_INH100153410,28,27,0,0,1,28,27,0,0,1,0,valid,Dum Maro Dum (remix),Asha Bhosle,INH100153410,,DUM MARO DUM (HARE RAMA HARE KRISHNA),ASHA BHOSLE,INP100000010,ANAND BAKSHI
youtube_dsr__rpaIYGfBtNM,crawler_nrkcommercialdma_ftp__NONRO187939CD0008,27,30,0,0,0,27,30,0,0,0,0,invalid,I Am Not a Robot,Marina And The Diamonds,,Marina Lambrini Diamandis,I AM NOT A ROBOT (STARSMITH 24 CARAT REMIX),MARINA AND THE DIAMONDS,GBFFS0900041,
youtube_dsr__sc3wUsa-pts,crawler_believe__12793215,13,12,0,0,0,13,12,0,0,0,0,invalid,Monster (Acoustic Version) [Paramore Cover],Bar Lounge,FR0W61761123,Hayley Nichole Williams|Taylor Benjamin York,MONSTER (LIVE FROM ITUNES SESSIONS),SKILLET,USAT21002429,
